# Holiday Gift Exchange

Like so many other things in 2020, our annual family gift exchange will be distupted.  Our tradition normally has us drawing names out of a hat at Thanksgiving, however this year were were unable to do so, for obvious reasons.

Given that I have email addresses for everybody in the family, I thought that automating the gift exchange process might be a fun little exercise!

## The Details

Normally, we place everybody's name in a hat and then family members take turns drawing, without replacement.  Drawn names are kept secere so that there is an extra element of surprise when the gifts are exchanged - the gift itself AND the gift-giver.  There are a few complications with this:

* Inevitably, people end up drawing their own name from the hat, in which case a replacement / redraw is allowed.
* Occasionally we'll have people draw the same name 2 or 3 years in a row.  Some people mind this and others don't.
* In some years, we'll have new memebers joining the exchange.  New entrants into the tradition may not be familiar enough with the entire family to be comfortable buying for any other participant.  In such cases, we want to constrain the pool from which they can draw.

## The Task

Write a script that takes the following inputs:

* Name
* Email
* Previous Year Selection (optional)
* Exclusion Names (optional)

Given the above, the script randomly assigns a selection to each entrant subject to the following constraints:

* Participants cannot be assigned to buy a gift for themselves
* Participants who submit a "previous year selection" cannot be assigned their pervious year selection
* Participants who submit a list of "Exclustion Names" may not be assigned names from their list of exclusions

Once selected, each entrent is sent an email with their assigned name and the parameters of the gift exchange (i.e. $ limit etc)

Also of note is that this task is **not** computationally intensive, nor is expected to become intensive in the coming years.  As such, we'll aim for a relatively simple solution.


## The Code

I've decided to store all the details in a config file to facilitate easier sharing of this little project.


In [1]:
import configparser

#get all the important params from the config
config = configparser.ConfigParser()
config.read('C:/Users/Paul/Documents/giftExchange_example.ini')
config.sections()

PARTICIPANTS= config['PATHS']['participants']
OUTPUTPATH = config['PATHS']['output']

USER = config['EMAIL']['user'] 
PWD = config['EMAIL']['pwd'] 

LIMIT = config['PARAMETERS']['limit']


For simplicity, input data will be stored in an excel file and loaded into pandas.  We'll load our test data and take a look:

In [2]:
import pandas as pd
import numpy as np
from collections import Iterable
import random 

#get participant data from excel
df = pd.read_excel(PARTICIPANTS,
                  headers=True,
                  index_col =  'Name')

df['Exclude'] = df['Exclude'].str.strip('()').str.split('|')


df

,Email,PrevYearSelection,Exclude
Name,,,
Alice,Alice@fakeDomain.Com,Bob,NaN
Bob,Bob@fakeDomain.Com,NaN,NaN
Charlie,Charlie@fakeDomain.Com,NaN,"[Dan, Bob]"
Dan,Dan@fakeDomain.Com,NaN,NaN
Eve,Eve@fakeDomain.Com,Dan,[Alice]


The data looks good.  The next step is to build a list of exclusions to represent ineligible selections for each "Name"

In [3]:
#create a col with the index values so we can add them to exclusions 
df['self'] = df.index

#concat all the exculsions into a single list

df['exclusions'] = df[['self','PrevYearSelection','Exclude']].values.tolist()

#drop the extra columns
df.drop(['self','PrevYearSelection','Exclude'],axis=1,inplace=True)

#remove nans from the exclustion cols
df['exclusions'] = df['exclusions'].apply(lambda x: [i for i in x if str(i) != "nan"])


#quick function to un-nest lists in a df column
def flatten(nested):
    for i in nested:
        if isinstance(i, Iterable) and not isinstance(i, str):
            yield from flatten(i)
        else:
            yield i
            
#flatten the list
df['exclusions'] = [*map(list, map(flatten, df['exclusions'].values))]
            
            
df

,Email,exclusions
Name,,
Alice,Alice@fakeDomain.Com,"[Alice, Bob]"
Bob,Bob@fakeDomain.Com,[Bob]
Charlie,Charlie@fakeDomain.Com,"[Charlie, Dan, Bob]"
Dan,Dan@fakeDomain.Com,[Dan]
Eve,Eve@fakeDomain.Com,"[Eve, Dan, Alice]"


Now we have a dataframe which contains each participant, their email and a list of names that are not eligible for match.  We'll short the list according to the lengh of exclusions under the simple assumption that they'll be the hardest conditions to satisfy.

In [4]:
df['len'] = df['exclusions'].str.len()

df.sort_values(by='len',
               ascending=False,
               inplace=True)
df

,Email,exclusions,len
Name,,,
Charlie,Charlie@fakeDomain.Com,"[Charlie, Dan, Bob]",3
Eve,Eve@fakeDomain.Com,"[Eve, Dan, Alice]",3
Alice,Alice@fakeDomain.Com,"[Alice, Bob]",2
Bob,Bob@fakeDomain.Com,[Bob],1
Dan,Dan@fakeDomain.Com,[Dan],1


In [5]:
def generateGiftExchange(df):
    '''
    the funciton that does the actual selection
    '''

    #check error conditions
    #while any of these are False, we continue
    errors = False
    duplicates = False
    blanks = False
    
    while errors or duplicates or blanks is False:
        
        #create empty selection column and a list of eligible names
        df['selection'] = "" 
        remainingNames = df.index.tolist()
    
    
        #loop over each row of the df
        for idx,row in df.iterrows():
            
            #define the eligible selections for each participant
            eligible = list(set(remainingNames) - set(row.exclusions))
            
            #try to randomly make a selection
            #there is path dependence here so this may fail sometimes... 
            #but the task is small so we can just re-try
            try:
                selected = random.choice(eligible)
            except:
                break
            
            #if we get a selection remove the name from the list
            remainingNames.remove(selected)
            
            df.loc[idx,'selection'] = selected
    
        #check/update the error conditions
        errors = df.apply(lambda x: x['selection'] in x['exclusions'],axis=1).any() 
        duplicates = df['selection'].duplicated().any()
        blanks = (df.selection != '').all()
    
    return df

df = generateGiftExchange(df)


    
df

,Email,exclusions,len,selection
Name,,,,
Charlie,Charlie@fakeDomain.Com,"[Charlie, Dan, Bob]",3,Alice
Eve,Eve@fakeDomain.Com,"[Eve, Dan, Alice]",3,Charlie
Alice,Alice@fakeDomain.Com,"[Alice, Bob]",2,Dan
Bob,Bob@fakeDomain.Com,[Bob],1,Eve
Dan,Dan@fakeDomain.Com,[Dan],1,Bob


We can see from the above that we now have a an exchange selection for each participant and all constraints have been satisfied.

One final complicaiton is that participants often tend to forget their selection so we'll also keep a copy of the above in a safe place in case we need to re-visit it.  The intention is to only look at if absolutely required.  In all likelyhood, if we do have a situation where the historical list needs to be accessed, I'll likely write some code to allow the sending of a "refresher" email to the uncertain participant without my having to know anything about their selection.


# Communicating the Results

The easiest way to get the info to people is via email.  What we'll do here is:

* A generic email announcing participation and high-level details.  Also, participants can respond to this email with their "wish lists"
* A specific email to each participant with the details about their pick and the spending limit.

Below we have code that does this using the above dataframe and config file as input.



In [6]:
import smtplib

gmail_user = USER
gmail_pwd = PWD
#FROM = USER
SUBJECT = "Gift Exchange! General Email"


def sendMail(gmail_user,gmail_pwd,FROM,TO,SUBJECT,TEXT):
    # Prepare actual message
    message = """\From: %s\nTo: %s\nSubject: %s\n\n%s
    """ % (FROM, ", ".join(TO), SUBJECT, TEXT)

    try:
        server = smtplib.SMTP_SSL("smtp.gmail.com", 465) 
        server.login(gmail_user, gmail_pwd)
        server.sendmail(FROM, TO, message)
        #server.quit()
        server.close()
        print('successfully sent the mail')
    except:
        print("failed to send mail")


text = """Greetings,

YOUR GIFT EXCHANGE MESSAGE GOES HERE!

"""


individualMessage = """Greetings {},
         
Your Gift exchange selection is: {} \n

Remember, the limit this year is {} \n
                    
Happy Holidays!
                    
The Exchange-O-Matic-9000 System.

"""


### --- UNCOMMENT EMAIL CODE AS NEEDED

#send a general email to the entire list

#sendMail(gmail_user,
#             gmail_pwd,
#             FROM,
#             df.Email.tolist(),
#             SUBJECT,
#             text)
    

for idx,row in df.iterrows():
    print("")
    print("------")  
    print("emailing:",idx)  
    print(individualMessage.format(idx,row.selection,LIMIT))
    print("------") 
    print("")
    
#    sendMail(gmail_user,
#             gmail_pwd,
#             FROM,
#             [row.Email],
#             "gift exchange selection for {} ".format(idx),
#             individualMessage.format(idx,row.selection,LIMIT))
        
        


------
emailing: Charlie
Greetings Charlie,
         
Your Gift exchange selection is: Alice 


Remember, the limit this year is $100 

                    
Happy Holidays!
                    
The Exchange-O-Matic-9000 System.


------


------
emailing: Eve
Greetings Eve,
         
Your Gift exchange selection is: Charlie 


Remember, the limit this year is $100 

                    
Happy Holidays!
                    
The Exchange-O-Matic-9000 System.


------


------
emailing: Alice
Greetings Alice,
         
Your Gift exchange selection is: Dan 


Remember, the limit this year is $100 

                    
Happy Holidays!
                    
The Exchange-O-Matic-9000 System.


------


------
emailing: Bob
Greetings Bob,
         
Your Gift exchange selection is: Eve 


Remember, the limit this year is $100 

                    
Happy Holidays!
                    
The Exchange-O-Matic-9000 System.


------


------
emailing: Dan
Greetings Dan,
         
Your Gift exchange 